# X-ray Spectroscopy Data: Exploratory Data Analysis

This notebook performs EDA on X-ray Absorption Spectroscopy (XAS) data,
encompassing both XANES (X-ray Absorption Near-Edge Structure) and EXAFS
(Extended X-ray Absorption Fine Structure) analysis. XAS measurements at APS
beamlines probe elemental oxidation states through the near-edge region and
local atomic structure (coordination number, bond distances, disorder) through
the extended fine structure oscillations.

XANES/EXAFS data are typically collected at sector-specific beamlines such as
20-BM (bending magnet, general-purpose XAS), 10-ID (insertion device, high flux),
and 9-BM (high-throughput XAS). Data consist of energy-dependent absorption
measurements: the incident beam intensity I0 and transmitted/fluorescence
signals that yield the absorption coefficient µ(E).

**Prerequisites**: `pip install h5py numpy matplotlib scipy`

**Note**: For production XAS analysis, consider [Larch](https://xraypy.github.io/xraylarch/)
or [Athena/Artemis](https://bruceravel.github.io/demeter/).

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

# Larch concepts: in a full analysis pipeline you would use:
#   import larch
#   from larch.xafs import pre_edge, autobk, xftf
# Here we implement the core concepts from scratch for clarity.

plt.rcParams["figure.dpi"] = 120

In [ ]:
# -------------------------------------------------------------------
# Load XAS data
# Typical storage: energy array + raw detector channels (I0, It, If)
# Absorption coefficient mu(E) = ln(I0/It) for transmission mode
#                        mu(E) = If/I0    for fluorescence mode
# -------------------------------------------------------------------

FILEPATH = "xas_data.h5"

with h5py.File(FILEPATH, "r") as f:
    energy = f["/entry/instrument/monochromator/energy"][:]   # eV
    i0 = f["/entry/instrument/I0/data"][:]                    # incident beam
    it = f["/entry/instrument/It/data"][:]                    # transmitted beam
    # Fluorescence channel (if available)
    if "/entry/instrument/If/data" in f:
        i_fluor = f["/entry/instrument/If/data"][:]
        has_fluorescence = True
    else:
        has_fluorescence = False

# Compute absorption coefficient mu(E)
# Transmission mode: mu = ln(I0 / It)
mu_trans = np.log(np.clip(i0, 1, None) / np.clip(it, 1, None))

# Fluorescence mode: mu = If / I0
if has_fluorescence:
    mu_fluor = i_fluor / np.clip(i0, 1, None)

print(f"Energy range:  {energy[0]:.1f} to {energy[-1]:.1f} eV ({len(energy)} points)")
print(f"Energy step:   {np.median(np.diff(energy)):.2f} eV (median)")
print(f"I0 range:      [{i0.min():.0f}, {i0.max():.0f}]")
print(f"It range:      [{it.min():.0f}, {it.max():.0f}]")
print(f"mu(E) range:   [{mu_trans.min():.4f}, {mu_trans.max():.4f}]")
if has_fluorescence:
    print(f"If range:      [{i_fluor.min():.0f}, {i_fluor.max():.0f}]")

## Raw Spectrum Visualization

We plot the raw XAS spectrum (µ(E) vs energy) and identify the absorption edge.
The edge energy E0 is defined as the maximum of the first derivative of µ(E)
and corresponds to the excitation energy of a core electron.

In [ ]:
# Plot raw XAS spectrum and identify absorption edge

# Smooth the first derivative to find the edge energy E0
dmu = np.gradient(mu_trans, energy)
dmu_smooth = savgol_filter(dmu, window_length=11, polyorder=3)
e0_idx = np.argmax(dmu_smooth)
e0 = energy[e0_idx]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw mu(E) spectrum
axes[0].plot(energy, mu_trans, "b-", lw=1.0, label="\u03bc(E) transmission")
axes[0].axvline(e0, color="red", ls="--", lw=1, label=f"E0 = {e0:.1f} eV")
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("\u03bc(E)")
axes[0].set_title("Raw XAS Spectrum")
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3)

# First derivative -- edge identification
axes[1].plot(energy, dmu_smooth, "g-", lw=1.0, label="d\u03bc/dE (smoothed)")
axes[1].axvline(e0, color="red", ls="--", lw=1, label=f"E0 = {e0:.1f} eV")
axes[1].set_xlabel("Energy (eV)")
axes[1].set_ylabel("d\u03bc/dE")
axes[1].set_title("First Derivative (Edge Identification)")
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Edge energy E0: {e0:.2f} eV")
print(f"Edge jump:      {mu_trans[e0_idx + 50] - mu_trans[e0_idx - 50]:.4f}")

In [ ]:
# Plot I0 and transmission/fluorescence channels separately
# This helps diagnose detector issues, beam instabilities, and saturation

n_channels = 3 if has_fluorescence else 2
fig, axes = plt.subplots(1, n_channels, figsize=(6 * n_channels, 5))

# I0 -- incident beam intensity
axes[0].plot(energy, i0, "b-", lw=0.8)
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("I0 (counts)")
axes[0].set_title(f"I0 Channel (mean={i0.mean():.0f})")
axes[0].axvline(e0, color="red", ls="--", lw=0.8, alpha=0.5)
axes[0].grid(True, alpha=0.3)

# It -- transmitted beam
axes[1].plot(energy, it, "orange", lw=0.8)
axes[1].set_xlabel("Energy (eV)")
axes[1].set_ylabel("It (counts)")
axes[1].set_title(f"Transmission Channel (mean={it.mean():.0f})")
axes[1].axvline(e0, color="red", ls="--", lw=0.8, alpha=0.5)
axes[1].grid(True, alpha=0.3)

# If -- fluorescence (if available)
if has_fluorescence:
    axes[2].plot(energy, i_fluor, "g-", lw=0.8)
    axes[2].set_xlabel("Energy (eV)")
    axes[2].set_ylabel("If (counts)")
    axes[2].set_title(f"Fluorescence Channel (mean={i_fluor.mean():.0f})")
    axes[2].axvline(e0, color="red", ls="--", lw=0.8, alpha=0.5)
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check for saturation or dead counts
print(f"I0: min={i0.min():.0f}, max={i0.max():.0f}, zeros={np.sum(i0 == 0)}")
print(f"It: min={it.min():.0f}, max={it.max():.0f}, zeros={np.sum(it == 0)}")
if has_fluorescence:
    print(f"If: min={i_fluor.min():.0f}, max={i_fluor.max():.0f}, zeros={np.sum(i_fluor == 0)}")

## Pre-edge and Post-edge Analysis

Normalization is essential for comparing XANES spectra across different samples
and measurement conditions. The standard procedure is:

1. Fit a linear function to the **pre-edge** region (well below E0) to model
   the background absorption.
2. Fit a linear or quadratic function to the **post-edge** region (well above E0)
   to model the atomic absorption envelope.
3. The **edge step** is the difference between post-edge and pre-edge fits
   evaluated at E0.
4. Subtract the pre-edge and divide by the edge step to get the normalized
   µ(E), which goes from ~0 below the edge to ~1 above.

In [ ]:
# Background subtraction and edge step normalization

# Define energy regions relative to E0
e_rel = energy - e0

# Pre-edge region: -150 to -30 eV relative to E0
pre_mask = (e_rel >= -150) & (e_rel <= -30)
pre_coeffs = np.polyfit(energy[pre_mask], mu_trans[pre_mask], 1)
pre_line = np.polyval(pre_coeffs, energy)

# Post-edge region: +50 to +300 eV relative to E0
post_mask = (e_rel >= 50) & (e_rel <= 300)
post_coeffs = np.polyfit(energy[post_mask], mu_trans[post_mask], 2)  # quadratic
post_line = np.polyval(post_coeffs, energy)

# Edge step: difference between post-edge and pre-edge at E0
edge_step = np.polyval(post_coeffs, e0) - np.polyval(pre_coeffs, e0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Show pre/post-edge fits on the raw spectrum
axes[0].plot(energy, mu_trans, "b-", lw=1.0, label="\u03bc(E)")
axes[0].plot(energy, pre_line, "r--", lw=1.0, label="Pre-edge fit")
axes[0].plot(energy, post_line, "g--", lw=1.0, label="Post-edge fit")
axes[0].axvline(e0, color="gray", ls=":", lw=0.8)
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("\u03bc(E)")
axes[0].set_title(f"Pre/Post-edge Fitting (edge step = {edge_step:.4f})")
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

# Background-subtracted spectrum
mu_bgsub = mu_trans - pre_line
axes[1].plot(energy, mu_bgsub, "b-", lw=1.0)
axes[1].axhline(0, color="gray", ls="--", lw=0.5)
axes[1].axhline(edge_step, color="orange", ls="--", lw=1.0, label=f"Edge step = {edge_step:.4f}")
axes[1].axvline(e0, color="gray", ls=":", lw=0.8)
axes[1].set_xlabel("Energy (eV)")
axes[1].set_ylabel("\u03bc(E) \u2212 background")
axes[1].set_title("Background-Subtracted Spectrum")
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Pre-edge slope:  {pre_coeffs[0]:.6f} per eV")
print(f"Edge step:       {edge_step:.4f}")

In [ ]:
# Normalized XANES spectrum extraction
# mu_norm = (mu - pre_edge_line) / edge_step

mu_norm = (mu_trans - pre_line) / max(edge_step, 1e-10)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full normalized spectrum
axes[0].plot(energy, mu_norm, "b-", lw=1.0)
axes[0].axhline(0, color="gray", ls="--", lw=0.5)
axes[0].axhline(1, color="gray", ls="--", lw=0.5)
axes[0].axvline(e0, color="red", ls="--", lw=0.8, alpha=0.5)
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("Normalized \u03bc(E)")
axes[0].set_title("Normalized XANES Spectrum")
axes[0].grid(True, alpha=0.3)

# Zoom into the XANES region (E0 - 20 to E0 + 80 eV)
xanes_mask = (e_rel >= -20) & (e_rel <= 80)
axes[1].plot(energy[xanes_mask], mu_norm[xanes_mask], "b-", lw=1.2)
axes[1].axhline(0, color="gray", ls="--", lw=0.5)
axes[1].axhline(1, color="gray", ls="--", lw=0.5)
axes[1].axvline(e0, color="red", ls="--", lw=0.8, label=f"E0 = {e0:.1f} eV")
axes[1].set_xlabel("Energy (eV)")
axes[1].set_ylabel("Normalized \u03bc(E)")
axes[1].set_title("XANES Region (detail)")
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Quality check: pre-edge should be ~0, far post-edge should be ~1
pre_check = np.mean(mu_norm[pre_mask])
post_check = np.mean(mu_norm[post_mask])
print(f"Mean normalized value in pre-edge region:  {pre_check:.4f} (should be ~0)")
print(f"Mean normalized value in post-edge region: {post_check:.4f} (should be ~1)")

## EXAFS Extraction

The EXAFS signal χ(k) represents oscillations in µ(E) above the absorption edge,
caused by interference of the outgoing photoelectron wave with waves
backscattered from neighboring atoms. The key steps are:

1. Convert energy to photoelectron wavenumber: k = sqrt(2m(E - E0) / hbar^2)
2. Subtract the smooth atomic background µ0(E) (using a spline)
3. Normalize by the edge step to get χ(k)
4. Apply k-weighting (k^n * χ(k)) to compensate for amplitude decay
5. Fourier transform to R-space to obtain the radial distribution function

In [ ]:
# Convert energy to k-space (photoelectron wavenumber)
# k = sqrt(2 * m_e * (E - E0) / hbar^2)
# With E in eV, k in inverse Angstroms: k = 0.5123 * sqrt(E - E0)

KCONV = 0.5123  # sqrt(2 * m_e / hbar^2) in Angstrom^-1 eV^-1/2

# Only use data above E0
exafs_mask = energy > (e0 + 5)  # start 5 eV above edge
e_exafs = energy[exafs_mask]
mu_exafs = mu_trans[exafs_mask]

# Convert to k-space
k = KCONV * np.sqrt(e_exafs - e0)

# chi(k) extraction concept:
# In a full analysis, one would fit a spline to the post-edge mu0(E)
# and compute chi = (mu - mu0) / (edge_step * mu0_at_E0).
# Here we use a simple polynomial approximation for the background.

# Fit a smooth background to mu in k-space
from numpy.polynomial.chebyshev import chebfit, chebval
bg_coeffs = chebfit(k, mu_exafs, deg=6)
mu_bg = chebval(k, bg_coeffs)

# Extract chi(k)
chi_k = (mu_exafs - mu_bg) / max(edge_step, 1e-10)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# mu(E) with background in EXAFS region
axes[0].plot(e_exafs, mu_exafs, "b-", lw=0.8, label="\u03bc(E)")
axes[0].plot(e_exafs, mu_bg, "r--", lw=1.0, label="Background \u03bc0(E)")
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("\u03bc(E)")
axes[0].set_title("EXAFS Region with Background")
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3)

# chi(k)
axes[1].plot(k, chi_k, "b-", lw=0.8)
axes[1].set_xlabel("k (\u00c5\u207b\u00b9)")
axes[1].set_ylabel("\u03c7(k)")
axes[1].set_title("Extracted \u03c7(k)")
axes[1].axhline(0, color="gray", ls="--", lw=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"k range: {k[0]:.2f} to {k[-1]:.2f} \u00c5\u207b\u00b9 ({len(k)} points)")

In [ ]:
# k-weighted chi(k) and Fourier transform to R-space

# k-weighting: multiply chi(k) by k^n (n=1,2,3) to compensate
# for the natural decay of EXAFS amplitude at high k
chi_k1 = k * chi_k          # k^1 weighted
chi_k2 = k**2 * chi_k       # k^2 weighted
chi_k3 = k**3 * chi_k       # k^3 weighted

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# k-weighted chi(k) plots
for i, (chi_kw, label, color) in enumerate([
    (chi_k1, "k\u00b9\u03c7(k)", "blue"),
    (chi_k2, "k\u00b2\u03c7(k)", "green"),
    (chi_k3, "k\u00b3\u03c7(k)", "red"),
]):
    axes[0].plot(k, chi_kw, color=color, lw=0.8, label=label)
axes[0].set_xlabel("k (\u00c5\u207b\u00b9)")
axes[0].set_ylabel("k\u207f\u03c7(k)")
axes[0].set_title("k-weighted \u03c7(k)")
axes[0].axhline(0, color="gray", ls="--", lw=0.5)
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

# Fourier transform to R-space
# Apply a Hanning window to reduce truncation artifacts
k_min, k_max = 2.0, min(k[-1], 14.0)  # typical k-range for FT
ft_mask = (k >= k_min) & (k <= k_max)
k_ft = k[ft_mask]
chi_ft = chi_k2[ft_mask]  # use k^2 weighting for FT

# Hanning window
window = np.hanning(len(k_ft))
chi_windowed = chi_ft * window

# Compute FT: chi(R) = integral of chi(k) * exp(2ikR) dk
# Use zero-padded FFT for interpolation
nfft = 2048
dk = np.mean(np.diff(k_ft))
chi_padded = np.zeros(nfft)
chi_padded[:len(chi_windowed)] = chi_windowed

ft_result = np.fft.fft(chi_padded) * dk
r_axis = np.fft.fftfreq(nfft, d=dk) * np.pi  # R in Angstroms

# Take positive R values only
pos_mask = (r_axis > 0) & (r_axis < 6)
r_pos = r_axis[pos_mask]
ft_magnitude = np.abs(ft_result[pos_mask])

axes[1].plot(k_ft, chi_ft, "b-", lw=0.8, alpha=0.5, label="\u03c7(k)")
axes[1].plot(k_ft, chi_windowed, "r-", lw=1.0, label="Windowed")
axes[1].set_xlabel("k (\u00c5\u207b\u00b9)")
axes[1].set_ylabel("k\u00b2\u03c7(k)")
axes[1].set_title(f"FT Window (k = {k_min:.1f}\u2013{k_max:.1f} \u00c5\u207b\u00b9)")
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

# R-space magnitude (radial distribution)
axes[2].plot(r_pos, ft_magnitude, "b-", lw=1.2)
axes[2].set_xlabel("R (\u00c5)")
axes[2].set_ylabel("|\u03c7(R)| (\u00c5\u207b\u00b3)")
axes[2].set_title("Fourier Transform Magnitude (R-space)")
axes[2].grid(True, alpha=0.3)

# Annotate expected first-shell distance (phase-shifted)
if len(ft_magnitude) > 0:
    peak_idx = np.argmax(ft_magnitude)
    peak_r = r_pos[peak_idx]
    axes[2].axvline(peak_r, color="red", ls="--", lw=0.8,
                    label=f"1st shell peak R = {peak_r:.2f} \u00c5")
    axes[2].legend(fontsize=8)

plt.tight_layout()
plt.show()

print(f"First shell peak (phase-uncorrected): R = {peak_r:.2f} \u00c5")
print("Note: True bond distance = R + ~0.2-0.5 \u00c5 (phase correction)")

## Multi-spectrum Analysis

Comparing spectra from different samples, oxidation states, or experimental
conditions reveals chemical changes. Linear combination fitting (LCF) uses
reference spectra of known standards to determine the fractional composition
of an unknown mixture.

In [ ]:
# Compare multiple spectra from different samples or conditions
# and demonstrate linear combination fitting (LCF) concept

# In practice, load multiple spectra from separate files or HDF5 groups.
# Here we demonstrate the concept with the available data and synthetic references.

# Simulate reference spectra for two end-member standards
# (e.g., reduced vs. oxidized states of the same element)
np.random.seed(42)
ref1_norm = mu_norm + 0.05 * np.sin(0.1 * (energy - e0))  # Reference A (e.g., Fe2+)
ref2_norm = mu_norm - 0.08 * np.sin(0.1 * (energy - e0)) + 0.1  # Reference B (e.g., Fe3+)

# Create a "mixture" spectrum as known linear combination
true_frac_a = 0.6
mixture = true_frac_a * ref1_norm + (1 - true_frac_a) * ref2_norm
mixture += 0.005 * np.random.randn(len(energy))  # add noise

# Linear combination fitting: find fractions that minimize residual
# mixture ~ a * ref1 + (1-a) * ref2
# Fit in the XANES region near the edge
fit_mask = (e_rel >= -20) & (e_rel <= 60)

def lcf_model(energy_fit, frac_a):
    """Linear combination of two reference spectra."""
    return frac_a * ref1_norm[fit_mask] + (1 - frac_a) * ref2_norm[fit_mask]

popt, pcov = curve_fit(lcf_model, energy[fit_mask], mixture[fit_mask],
                       p0=[0.5], bounds=(0, 1))
fit_frac_a = popt[0]
fit_result = fit_frac_a * ref1_norm + (1 - fit_frac_a) * ref2_norm
residual = mixture - fit_result

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Reference spectra comparison
axes[0].plot(energy, ref1_norm, "b-", lw=1.0, label="Reference A (reduced)")
axes[0].plot(energy, ref2_norm, "r-", lw=1.0, label="Reference B (oxidized)")
axes[0].plot(energy, mixture, "k--", lw=1.0, alpha=0.7, label="Mixture (unknown)")
axes[0].set_xlabel("Energy (eV)")
axes[0].set_ylabel("Normalized \u03bc(E)")
axes[0].set_title("Multi-spectrum Comparison")
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

# LCF fit result
axes[1].plot(energy[fit_mask], mixture[fit_mask], "ko", ms=2, label="Data")
axes[1].plot(energy[fit_mask], fit_result[fit_mask], "r-", lw=1.5, label="LCF fit")
axes[1].set_xlabel("Energy (eV)")
axes[1].set_ylabel("Normalized \u03bc(E)")
axes[1].set_title(f"LCF: {fit_frac_a:.1%} A + {1-fit_frac_a:.1%} B (true: {true_frac_a:.1%}/{1-true_frac_a:.1%})")
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

# Fit residual
axes[2].plot(energy, residual, "g-", lw=0.8)
axes[2].axhline(0, color="gray", ls="--", lw=0.5)
axes[2].set_xlabel("Energy (eV)")
axes[2].set_ylabel("Residual")
rms = np.sqrt(np.mean(residual[fit_mask]**2))
axes[2].set_title(f"LCF Residual (RMS = {rms:.4f})")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"LCF result:  {fit_frac_a:.3f} Reference A + {1-fit_frac_a:.3f} Reference B")
print(f"True values: {true_frac_a:.3f} Reference A + {1-true_frac_a:.3f} Reference B")
print(f"Fit RMS:     {rms:.5f}")

## Spatial-Spectral Data

XANES imaging (also called spectro-microscopy or µ-XANES mapping) collects
a full XANES spectrum at each pixel of a 2D map. This produces a 3D datacube
(x, y, energy) that enables spatially-resolved chemical speciation. Edge energy
maps reveal the spatial distribution of oxidation states.

In [ ]:
# XANES mapping concept: spectra at each pixel, edge energy maps
#
# A XANES map dataset has shape (n_energies, n_rows, n_cols)
# At each (row, col) pixel, we have a full XANES spectrum.
# By fitting the edge position at each pixel, we create an edge energy map
# that shows the spatial distribution of oxidation states.

# Simulate a small XANES map for demonstration
np.random.seed(0)
map_rows, map_cols = 50, 60
n_energies = 80
energy_map = np.linspace(e0 - 20, e0 + 60, n_energies)

# Create spatially varying edge energy (simulating mixed oxidation states)
yy, xx = np.mgrid[:map_rows, :map_cols]
# Two "grains" with different oxidation states
grain1 = np.exp(-((yy - 15)**2 + (xx - 20)**2) / (2 * 8**2))
grain2 = np.exp(-((yy - 35)**2 + (xx - 40)**2) / (2 * 10**2))
e0_map = e0 + 2.0 * grain1 - 1.5 * grain2  # spatially varying E0

# Generate XANES datacube: spectrum at each pixel
xanes_cube = np.zeros((n_energies, map_rows, map_cols))
for r in range(map_rows):
    for c in range(map_cols):
        local_e0 = e0_map[r, c]
        # Simple arctan edge model
        xanes_cube[:, r, c] = 0.5 + 0.5 * np.tanh((energy_map - local_e0) / 2.0)
        # Add white line
        xanes_cube[:, r, c] += 0.2 * np.exp(-0.5 * ((energy_map - local_e0 - 2) / 1.5)**2)
        # Add noise
        xanes_cube[:, r, c] += 0.01 * np.random.randn(n_energies)

# Compute edge energy map from derivative maximum at each pixel
edge_map = np.zeros((map_rows, map_cols))
for r in range(map_rows):
    for c in range(map_cols):
        spectrum = xanes_cube[:, r, c]
        deriv = np.gradient(spectrum, energy_map)
        deriv_smooth = savgol_filter(deriv, min(11, len(deriv) // 2 * 2 + 1), 3)
        edge_map[r, c] = energy_map[np.argmax(deriv_smooth)]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Single-energy absorption map (at the edge)
e_at_edge_idx = np.argmin(np.abs(energy_map - e0))
im0 = axes[0].imshow(xanes_cube[e_at_edge_idx], cmap="viridis", origin="lower")
axes[0].set_title(f"Absorption Map at E = {energy_map[e_at_edge_idx]:.1f} eV")
axes[0].set_xlabel("X (pixel)")
axes[0].set_ylabel("Y (pixel)")
plt.colorbar(im0, ax=axes[0], fraction=0.046, label="\u03bc")

# Edge energy map (oxidation state distribution)
im1 = axes[1].imshow(edge_map, cmap="RdBu_r", origin="lower",
                     vmin=e0 - 3, vmax=e0 + 3)
axes[1].set_title("Edge Energy Map (oxidation state)")
axes[1].set_xlabel("X (pixel)")
axes[1].set_ylabel("Y (pixel)")
plt.colorbar(im1, ax=axes[1], fraction=0.046, label="E0 (eV)")

# Selected pixel spectra from different regions
pixels_to_show = [(15, 20), (35, 40), (5, 5)]  # (row, col)
colors = ["blue", "red", "green"]
for (r, c), color in zip(pixels_to_show, colors):
    axes[2].plot(energy_map, xanes_cube[:, r, c], color=color, lw=1.0,
                label=f"({r},{c}), E0={edge_map[r,c]:.1f} eV")
axes[2].set_xlabel("Energy (eV)")
axes[2].set_ylabel("Normalized \u03bc(E)")
axes[2].set_title("Selected Pixel Spectra")
axes[2].legend(fontsize=8)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"XANES map shape: {xanes_cube.shape} (energies x rows x cols)")
print(f"Edge energy range: {edge_map.min():.2f} to {edge_map.max():.2f} eV")
print(f"Edge energy spread: {edge_map.std():.3f} eV (std)")

## EDA Summary

After running this notebook, you should have assessed:

1. **Data loading** -- Loaded energy, I0, It/If channels from HDF5 and computed µ(E)
2. **Raw spectrum** -- Visualized µ(E) and identified the absorption edge energy E0
3. **Detector channels** -- Inspected I0 and transmission/fluorescence signals for quality
4. **Normalization** -- Applied pre-edge subtraction and edge step normalization
5. **XANES extraction** -- Obtained the normalized XANES spectrum
6. **EXAFS extraction** -- Converted to k-space, extracted χ(k), applied k-weighting
7. **R-space analysis** -- Fourier transformed to obtain the radial distribution function
8. **Multi-spectrum comparison** -- Compared spectra and demonstrated linear combination fitting
9. **Spatial-spectral data** -- Demonstrated XANES mapping and edge energy maps for
   spatially-resolved chemical speciation

**Next steps**: Quantitative EXAFS fitting (bond distances, coordination numbers,
Debye-Waller factors) using Larch or Artemis, PCA/clustering of XANES maps,
and correlation with complementary techniques (XRF, XRD).